<a href="https://colab.research.google.com/github/vsrivatsan2096/iku/blob/master/iku_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre-Trained embeddings

In [0]:
!wget "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2019-03-13 16:50:47--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.168.85
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.168.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  80.5MB/s    in 26s     

2019-03-13 16:51:13 (60.5 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [19]:
!wget "http://nlp.stanford.edu/data/glove.840B.300d.zip"

--2019-03-14 08:28:28--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2019-03-14 08:28:28--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘glove.840B.300d.zip’

glove.840B.300d.zip 100%[===================>]   2.03G  6.64MB/s    in 4m 17s  

2019-03-14 08:32:45 (8.07 MB/s) - ‘glove.840B.300d.zip’ saved [2176768927/2176768927]



In [0]:
import os
import zipfile

In [0]:
wiki_zip = zipfile.ZipFile('glove.840B.300d.zip')
wiki_zip.extractall()
os.remove("glove.840B.300d.zip")

In [11]:
!wget https://dl.fbaipublicfiles.com/infersent/infersent1.pkl

--2019-03-14 08:24:17--  https://dl.fbaipublicfiles.com/infersent/infersent1.pkl
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.20.6.166, 104.20.22.166, 2606:4700:10::6814:6a6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.20.6.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 154010676 (147M) [application/octet-stream]
Saving to: ‘infersent1.pkl’

infersent1.pkl      100%[===================>] 146.88M  45.0MB/s    in 3.3s    

2019-03-14 08:24:21 (45.0 MB/s) - ‘infersent1.pkl’ saved [154010676/154010676]



In [0]:
infersent_embeddings = 'infersent1.pkl'

In [0]:
google_embeddings = 'GoogleNews-vectors-negative300.bin.gz'

In [0]:
wiki_embeddings = 'glove.840B.300d.txt'

# Google Drive

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

    100% |████████████████████████████████| 993kB 9.9MB/s 


In [0]:
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1bqdQccb6176JhXU7CZbWnv3vTAi7BH_-'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('questions.csv')

# Mount Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Cloning Github

In [9]:
!git clone https://github.com/vsrivatsan2096/iku.git

fatal: destination path 'iku' already exists and is not an empty directory.


# Dataset

In [0]:
import numpy as np
import pandas as pd
import multiprocessing

In [0]:
questions = pd.read_csv("questions.csv")
questions.dropna(inplace=True)

In [0]:
questions1 = questions.iloc[:, 3].values
questions2 = questions.iloc[:, 4].values
is_duplicate_questions = questions.iloc[:, 5].values

In [0]:
length = is_duplicate_questions.shape[0]

# Scores

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import jaccard_similarity_score
from sklearn.neighbors import DistanceMetric

In [0]:
class Scores(object):
    def __init__(self, sentences1, sentences2, length):
        self.sentences1 = sentences1
        self.sentences2 = sentences2
        self.length = length
        self.scores = np.zeros((length, 4))
    
    def cosine_similarity(self, sentence1, sentence2):
        return cosine_similarity(sentence1.reshape(1, -1), sentence2.reshape(1, -1))
    
    def manhattan_distances(self, sentence1, sentence2):
        return manhattan_distances(sentence1.reshape(1, -1), sentence2.reshape(1, -1))
    
    def euclidean_distances(self, sentence1, sentence2):
        return euclidean_distances(sentence1.reshape(1, -1), sentence2.reshape(1, -1))
    
    def jaccard_similarity(self, sentence1, sentence2):
        jac_score = 0
        try:
            jac_score = jaccard_similarity_score(sentence1.reshape(1, -1), sentence2.reshape(1, -1))
        finally:
            return jac_score

    def minkowski_distances(self, sentence1, sentence2):
        minkowski_distance = DistanceMetric.get_metric('minkowski')
        return minkowski_distance.pairwise(sentence1.reshape(1, -1), sentence2.reshape(1, -1))

    
    def get_scores(self):
        scores = [['cosine similarity', 'manhattan distances', 'euclidean distances', 'jaccard similarity', ]]
        for each in range(self.length):
            each_score = []
            each_score.append(self.cosine_similarity(self.sentences1[each], self.sentences2[each])[0][0])
            each_score.append(self.manhattan_distances(self.sentences1[each], self.sentences2[each])[0][0])
            each_score.append(self.euclidean_distances(self.sentences1[each], self.sentences2[each])[0][0])
            #each_score.append(self.jaccard_similarity(self.sentences1[each], self.sentences2[each]))
            each_score.append(self.minkowski_distances(self.sentences1[each], self.sentences2[each])[0][0])
            self.scores[each] = np.asarray(each_score)
        return self.scores

# NLP Preprocessing

In [0]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [0]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
lemma = WordNetLemmatizer()
stopword = stopwords.words("english")

In [0]:
sentences1 = []
for i in questions1:
    tempx = re.sub(r"[^A-Za-z]", " ", str(i))
    tempx = tempx.lower().split()
    tempx = [word for word in tempx if word not in stopword]
    tempx = [lemma.lemmatize(word, pos="a") for word in tempx]
    tempx = [lemma.lemmatize(word, pos="r") for word in tempx]
    tempx = [lemma.lemmatize(word, pos="n") for word in tempx]
    tempx = [lemma.lemmatize(word, pos="v") for word in tempx]
    sentences1.append(" ".join(tempx))
sentences1 = np.asarray(sentences1)

In [0]:
sentences2 = []
for i in questions2:
    tempx = re.sub(r"[^A-Za-z]", " ", str(i))
    tempx = tempx.lower().split()
    tempx = [word for word in tempx if word not in stopword]
    tempx = [lemma.lemmatize(word, pos="a") for word in tempx]
    tempx = [lemma.lemmatize(word, pos="r") for word in tempx]
    tempx = [lemma.lemmatize(word, pos="n") for word in tempx]
    tempx = [lemma.lemmatize(word, pos="v") for word in tempx]
    sentences2.append(" ".join(tempx))
sentences2 = np.asarray(sentences2)

# LSA

In [0]:
from iku.methods.lsa.lsa import LatentSemanticAnalysis

In [0]:
lsa_method = LatentSemanticAnalysis(sentences1, sentences2)

In [0]:
lsa_1, lsa_2 = lsa_method.fit_transform()

In [0]:
scores = Scores(lsa_1, lsa_2, lsa_1.shape[0])

In [0]:
lsa_x = scores.get_scores()

In [0]:
lsa_x.shape

(404348, 4)

In [0]:
del lsa_1
del lsa_2
del lsa_method
del scores
del sentences1
del sentences2

# Sent2Vec

In [0]:
from gensim.models.keyedvectors import KeyedVectors

In [0]:
google_model = KeyedVectors.load_word2vec_format(google_embeddings, binary=True)

In [0]:
from iku.methods.sent2vec.sent2vec import sentence_vectorizer

In [0]:
sent_vec1 = []
for each in sentences1:
    temp = sentence_vectorizer(google_model, each)
    if temp.shape[0] != 0:
        sent_vec1.append(temp)
    else:
        sent_vec1.append(np.zeros((300,)))
sent_vec1 = np.asarray(sent_vec1)

In [0]:
sent_vec2 = []
for each in sentences2:
    temp = sentence_vectorizer(google_model, each)
    if temp.shape[0] != 0:
        sent_vec2.append(temp)
    else:
        sent_vec2.append(np.zeros((300,)))
sent_vec2 = np.asarray(sent_vec2)

In [0]:
scores = Scores(sent_vec1, sent_vec2, length)

In [0]:
sent2vec_x = scores.get_scores()

In [0]:
del sent_vec2
del scores
del google_model

# InferText model

In [0]:
import torch

In [0]:
from iku.methods.infersent.models import InferSent

In [0]:
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': 1}

In [0]:
model = InferSent(params_model)
model.load_state_dict(torch.load(infersent_embeddings))

In [0]:
use_cuda = False
model = model.cuda() if use_cuda else model # Keep it on CPU or put it on GPU

In [0]:
model.set_w2v_path(wiki_embeddings)

In [0]:
model.build_vocab_k_words(K=100000)

In [0]:
infersent_embeddings_1 = model.encode(sentences_1, bsize=128, tokenize=False, verbose=True)

In [0]:
infersent_embeddings_2 = model.encode(sentences_2, bsize=128, tokenize=False, verbose=True)


In [0]:
scores = Scores(infersent_embeddings_1, infersent_embeddings_2, length)

In [0]:
infersent_x = scores.get_scores()

In [0]:
del model
del infersent_embeddings_1
del infersent_embeddings_2
del scores